### Import necessary packages:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np
import astropy.io.fits as fits
import healpy as hp
from matplotlib import cm
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Galactic
import astropy.units as u

### Read in Dwingeloo data:

In [ ]:
hduQ = fits.open('/srv/data/dwingeloo/DWING_465_EQU_Q.HPX.fits')
hdrQ = hduQ[0].header
dataQ = hduQ[0].data[0]
print(dataQ.shape)

hduU = fits.open('/srv/data/dwingeloo/DWING_465_EQU_U.HPX.fits')
hdrU = hduU[0].header
dataU = hduU[0].data[0]
print(dataU.shape)

### Pick the range of RA and dec to plot, and the gridding pixel size

In [ ]:
# Sample a grid in RA/Dec
RA_min = 0
RA_max = 360
dec_min = -10
dec_max = 90
delta = 0.5

### Set up the HEALpix projection, grid and coordinates
#### NOTE: if all files have the same NSIDE, just need to do this once

In [ ]:
# Healpix parameters:
nside = hdrQ['NSIDE']
order = hdrQ['ORDERING']
hpx = HEALPix(nside=nside, order=order, frame=ICRS())

# Number of pixels along each axis:
numRA = int((RA_max-RA_min)/delta)
numdec = int((dec_max-dec_min)/delta)

# 1D and 2D RA and dec grids:
ra = np.linspace(RA_min, RA_max, numRA) * u.deg
dec = np.linspace(dec_min, dec_max, numdec) * u.deg
ra_grid, dec_grid = np.meshgrid(ra, dec)

# Set up Astropy coordinate objects
coords = SkyCoord(ra_grid.ravel(), dec_grid.ravel(), frame='icrs')

### Reproject the HEALpix values onto the 2D grid for each map:

In [ ]:
Qmap = hpx.interpolate_bilinear_skycoord(coords, dataQ)
Qmap = Qmap.reshape((numdec, numRA))[:,::-1]
print(Qmap.shape)

Umap = hpx.interpolate_bilinear_skycoord(coords, dataU)
Umap = Umap.reshape((numdec, numRA))[:,::-1]
print(Umap.shape)


### Plots:

In [ ]:
plt.figure(figsize=(20, 4))
im = plt.imshow(Qmap, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.RdBu_r, 
                origin='lower', aspect='equal',vmin=-1.5,vmax=1.5)
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('Stokes Q')
plt.show()

plt.figure(figsize=(20, 4))
im = plt.imshow(Umap, extent=[RA_max, RA_min, dec_min, dec_max], cmap=plt.cm.RdBu_r, 
                origin='lower', aspect='equal',vmin=-1.5,vmax=1.5)
plt.colorbar(im)
plt.xlabel('Right ascension (deg)')
plt.ylabel('Declination (deg)')
plt.title('Stokes U')
plt.show()